
# Learning Management System (LMS) (Brief Documentation)


## Objective:
We have tried to build a simple learning management system like (Brightspace of MUN) which is implemented in PyQT5. We have tried to build an educational environment which can manage users, courses, and communication within the environment similar to MUN Brightspace. We have used almost every design pattern to build the PyQT5-based app.


## Installation and Running the Application:
### Install PyQT5:
```bash
pip install pyqt5


In [1]:
pip install pyqt5

Note: you may need to restart the kernel to use updated packages.


Also We Have to open Jupyter Notebook and run file called Group24_Project.ipynb.

## Features and Functions of the Project:
### User Management:
#### Registration:
- We have to first enter the name of the user and select a role from ADMIN, STUDENT, TEACHER.
- Then we have to click the button called "Register User", it will create a new user in the system.

#### Login:
- After username registration, we have to write the username and then we have to click the "Login" button to work with the system.
### Course Management:
#### Create Course:
- Enter the name of the course and press the "Create Course" button to create a course. (Only Teachers can do it.)

#### Enroll in Course:
- Enter the name of the course and press the "Enroll in Course" button to enroll in a course. (Only Students can do it.)

#### Add Course Content:
- Input the course name and upload the file of the content you want to add to the course, and click the "Add Content to Course" button. (Only for Teachers.)

#### View Course Content:
- Input the course name and click the "View Content" button. It will open the content list and then you can

### Communication:
#### Send Message:
- Input the receiver’s username and the content of the message, and then click the "Send Message" button.

#### Read Message:
- You can see all the messages after clicking on this button.

#### Logout:
- It will log out from the current session.
 select it.



## Design Patterns Used:
- **Factory Pattern**: For User Creation, all three roles are created using this pattern (Teacher, Student, Admin).
- **Singleton Pattern**: Used for managing particular sessions. For example, if one user starts the system and logs out, one session has been done. So, it manages that.
- **Composite Pattern**: Manages course components.
- **Observer Pattern**: Manages notifications.
- **Strategy Pattern**: Content Delivery managed by this pattern.
- **Mediator Pattern**: Manages communication between users.
- **Template Pattern**: Used for assessments.
- **Command Pattern**: Handles operations in the system (e.g., Add, View Content).
## Data Persistence:
- All data is stored in `ims_data.json`, it has methods to save and load data.


In [2]:
import sys
import json
import os
import warnings
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QLabel, QPushButton, QLineEdit, QComboBox, QListWidget, QFileDialog, QVBoxLayout, QHBoxLayout, QMessageBox
from PyQt5.QtGui import QFont

In [3]:
# This line is mentioed specially because deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*sipPyTypeDict.*")

In [4]:
# For Uploading files.
upload_dir = 'uploads'
if not os.path.exists(upload_dir):
    os.makedirs(upload_dir)

In [5]:
# (Admin, Teacher, Student)
class User:
    def __init__(self, username, role):
        self.username = username
        self.role = role

class Admin(User):
    def __init__(self, username):
        super().__init__(username, 'Admin')

class Teacher(User):
    def __init__(self, username):
        super().__init__(username, 'Teacher')

class Student(User):
    def __init__(self, username):
        super().__init__(username, 'Student')

class UserFactory:
    @staticmethod
    def create_user(username, role):
        if role == 'Admin':
            return Admin(username)
        elif role == 'Teacher':
            return Teacher(username)
        elif role == 'Student':
            return Student(username)
        else:
            raise ValueError("Invalid role")

In [6]:
# Singleton Pattern :- For session management it ensures single instance of the session.
#Login Logout
class Session:
    instance = None

    def __new__(cls):
        if cls.instance is None:
            cls.instance = super(Session, cls).__new__(cls)
            cls.instance.user = None
        return cls.instance

    def login(self, user):
        self.user = user

    def logout(self):
        self.user = None

    def get_logged_in_user(self):
        return self.user

In [7]:
# Composite Pattern :- It is for managining course component. 
#(Courses, Modules, Lessons)
class Component:
    def show_details(self):
        pass

class Course(Component):
    def __init__(self, name):
        self.name = name
        self.components = []
        self.observers = []
        self.content = []

    def add_component(self, component):
        self.components.append(component)

    def remove_component(self, component):
        self.components.remove(component)

    def add_content(self, content):
        self.content.append(content)

    def show_details(self):
        print(f"Course: {self.name}")
        for component in self.components:
            component.show_details()
        for content in self.content:
            print(f"  Content: {content}")

    # Observer Pattern methods
    def add_observer(self, observer):
        self.observers.append(observer)

    def remove_observer(self, observer):
        self.observers.remove(observer)

    def notify_observers(self, message):
        for observer in self.observers:
            observer.update(message)

class Module(Component):
    def __init__(self, name):
        self.name = name
        self.lessons = []

    def add_lesson(self, lesson):
        self.lessons.append(lesson)

    def remove_lesson(self, lesson):
        self.lessons.remove(lesson)

    def show_details(self):
        print(f"  Module: {self.name}")
        for lesson in self.lessons:
            lesson.show_details()

class Lesson(Component):
    def __init__(self, name):
        self.name = name

    def show_details(self):
        print(f"    Lesson: {self.name}")

In [8]:
# Observer Pattern :- It manages notification in system.
class StudentObserver:
    def __init__(self, name):
        self.name = name

    def update(self, message):
        print(f"{self.name} received notification: {message}")

In [9]:
# Strategy Pattern :- Deliver course content in a system. (In differnt file formats.)
class ContentDeliveryStrategy:
    def deliver(self, content):
        pass

class VideoStrategy(ContentDeliveryStrategy):
    def deliver(self, content):
        print(f"Delivering video content: {content}")

class DocumentStrategy(ContentDeliveryStrategy):
    def deliver(self, content):
        print(f"Delivering document content: {content}")

class QuizStrategy(ContentDeliveryStrategy):
    def deliver(self, content):
        print(f"Delivering quiz content: {content}")

class ContentDeliveryContext:
    def __init__(self, strategy):
        self.strategy = strategy

    def set_strategy(self, strategy):
        self.strategy = strategy

    def deliver_content(self, content):
        self.strategy.deliver(content)

In [10]:
# Mediator Pattern :- It manages communication for users.
class Mediator:
    def send_message(self, sender, receiver, message):
        pass

class ChatMediator(Mediator):
    def __init__(self):
        self.participants = {}
        self.messages = []

    def register(self, user):
        self.participants[user.username] = user

    def send_message(self, sender, receiver, message):
        if receiver in self.participants:
            self.messages.append((sender, receiver, message, str(datetime.now())))
            print(f"{sender} to {receiver}: {message}")
        else:
            print(f"{receiver} not found")

    def get_messages_for_user(self, username):
        return [msg for msg in self.messages if msg[1] == username]

In [11]:
# Template Pattern :- It is used for assesment
class AssessmentTemplate:
    def create_assessment(self):
        self.create_questions()
        self.create_answers()
        self.grade_assessment()

    def create_questions(self):
        pass

    def create_answers(self):
        pass

    def grade_assessment(self):
        pass

class Quiz(AssessmentTemplate):
    def create_questions(self):
        print("Creating quiz questions")

    def create_answers(self):
        print("Creating quiz answers")

    def grade_assessment(self):
        print("Grading quiz")


In [12]:
# Command Pattern :- It executes system operations for example add and view content.
class Command:
    def execute(self):
        pass

class RegisterUserCommand(Command):
    def __init__(self, user_factory, username, role):
        self.user_factory = user_factory
        self.username = username
        self.role = role

    def execute(self):
        user = self.user_factory.create_user(self.username, self.role)
        print(f"User {user.username} registered as {user.role}")
        return user

class EnrollCourseCommand(Command):
    def __init__(self, course, student):
        self.course = course
        self.student = student

    def execute(self):
        print(f"{self.student.username} enrolled in {self.course.name}")


In [13]:
# Data Persistence :- 
class DataStore:
    def __init__(self, filename='lms_data.json'):
        self.filename = filename
        self.data = self.load_data()

    def load_data(self):
        try:
            with open(self.filename, 'r') as file:
                return json.load(file)
        except FileNotFoundError:
            return {"users": [], "courses": [], "messages": []}

    def save_data(self):
        with open(self.filename, 'w') as file:
            json.dump(self.data, file, indent=4)

    def add_user(self, user):
        self.data['users'].append({'username': user.username, 'role': user.role})
        self.save_data()

    def get_users(self):
        return self.data.get('users', [])

    def add_course(self, course):
        for saved_course in self.data['courses']:
            if saved_course['name'] == course.name:
                saved_course['components'] = [component.name for component in course.components]
                saved_course['content'] = course.content
                self.save_data()
                return
        self.data['courses'].append({'name': course.name, 'components': [component.name for component in course.components], 'content': course.content})
        self.save_data()

    def get_courses(self):
        return self.data.get('courses', [])

    def add_message(self, sender, receiver, message):
        self.data['messages'].append({'sender': sender, 'receiver': receiver, 'message': message, 'timestamp': str(datetime.now())})
        self.save_data()

    def get_messages_for_user(self, username):
        return [msg for msg in self.data['messages'] if msg['receiver'] == username]



In [14]:
# Function to upload a file and return its path
def upload_file():
    file_path, _ = QFileDialog.getOpenFileName()
    if not file_path:
        QMessageBox.warning(None, "No file selected", "No file was selected.")
        return None
    filename = os.path.basename(file_path)
    destination = os.path.join(upload_dir, filename)
    with open(file_path, 'rb') as src, open(destination, 'wb') as dst:
        dst.write(src.read())
    return destination


In [15]:
# Function to open a file
def open_file(file_path):
    if os.path.exists(file_path):
        os.startfile(file_path)
    else:
        QMessageBox.warning(None, "File Not Found", f"File not found: {file_path}")

class LMSApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Learning Management System")
        self.setGeometry(100, 100, 800, 600)

        # Initialize data store, user factory, session, and chat mediator
        self.data_store = DataStore()
        self.user_factory = UserFactory()
        self.session = Session()
        self.chat_mediator = ChatMediator()

        # Initialize UI components
        self.init_ui()

    def init_ui(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)

        # User Management
        user_layout = QVBoxLayout()
        user_label = QLabel("User Management")
        user_label.setFont(QFont('Arial', 16))
        user_layout.addWidget(user_label)

        self.username_input = QLineEdit(self)
        self.username_input.setPlaceholderText("Username")
        self.username_input.setFont(QFont('Arial', 12))
        user_layout.addWidget(self.username_input)

        self.role_input = QComboBox(self)
        self.role_input.addItems(["Admin", "Teacher", "Student"])
        self.role_input.setFont(QFont('Arial', 12))
        user_layout.addWidget(self.role_input)

        self.register_button = QPushButton("Register User", self)
        self.register_button.setFont(QFont('Arial', 12))
        self.register_button.clicked.connect(self.register_user)
        user_layout.addWidget(self.register_button)

        self.login_button = QPushButton("Login", self)
        self.login_button.setFont(QFont('Arial', 12))
        self.login_button.clicked.connect(self.login)
        user_layout.addWidget(self.login_button)

        layout.addLayout(user_layout)

        # Course Management
        course_layout = QVBoxLayout()
        course_label = QLabel("Course Management")
        course_label.setFont(QFont('Arial', 14))
        course_layout.addWidget(course_label)

        self.course_name_input = QLineEdit(self)
        self.course_name_input.setPlaceholderText("Course Name")
        self.course_name_input.setFont(QFont('Arial', 12))
        course_layout.addWidget(self.course_name_input)

        self.create_course_button = QPushButton("Create Course", self)
        self.create_course_button.setFont(QFont('Arial', 12))
        self.create_course_button.clicked.connect(self.create_course)
        course_layout.addWidget(self.create_course_button)

        self.enroll_course_button = QPushButton("Enroll in Course", self)
        self.enroll_course_button.setFont(QFont('Arial', 12))
        self.enroll_course_button.clicked.connect(self.enroll_in_course)
        course_layout.addWidget(self.enroll_course_button)

        self.add_content_button = QPushButton("Add Content to Course", self)
        self.add_content_button.setFont(QFont('Arial', 12))
        self.add_content_button.clicked.connect(self.add_content_to_course)
        course_layout.addWidget(self.add_content_button)

        self.view_content_button = QPushButton("View Content", self)
        self.view_content_button.setFont(QFont('Arial', 12))
        self.view_content_button.clicked.connect(self.view_content)
        course_layout.addWidget(self.view_content_button)

        layout.addLayout(course_layout)

        # Communication
        comm_layout = QVBoxLayout()
        comm_label = QLabel("Communication")
        comm_label.setFont(QFont('Arial', 14))
        comm_layout.addWidget(comm_label)

        self.receiver_input = QLineEdit(self)
        self.receiver_input.setPlaceholderText("Receiver")
        self.receiver_input.setFont(QFont('Arial', 12))
        comm_layout.addWidget(self.receiver_input)

        self.message_input = QLineEdit(self)
        self.message_input.setPlaceholderText("Message")
        self.message_input.setFont(QFont('Arial', 12))
        comm_layout.addWidget(self.message_input)

        self.send_message_button = QPushButton("Send Message", self)
        self.send_message_button.setFont(QFont('Arial', 12))
        self.send_message_button.clicked.connect(self.send_message)
        comm_layout.addWidget(self.send_message_button)

        self.read_messages_button = QPushButton("Read Messages", self)
        self.read_messages_button.setFont(QFont('Arial', 12))
        self.read_messages_button.clicked.connect(self.read_messages)
        comm_layout.addWidget(self.read_messages_button)

        self.logout_button = QPushButton("Logout", self)
        self.logout_button.setFont(QFont('Arial', 12))
        self.logout_button.clicked.connect(self.logout)
        comm_layout.addWidget(self.logout_button)

        layout.addLayout(comm_layout)

        # Messages list
        self.msg_list = QListWidget(self)
        self.msg_list.setFont(QFont('Arial', 12))
        layout.addWidget(QLabel("Messages"))
        layout.addWidget(self.msg_list)

        # Content list
        self.content_list = QListWidget(self)
        self.content_list.setFont(QFont('Arial', 12))
        layout.addWidget(QLabel("Content"))
        layout.addWidget(self.content_list)
        self.open_content_button = QPushButton("Open Selected Content", self)
        self.open_content_button.setFont(QFont('Arial', 12))
        self.open_content_button.clicked.connect(self.open_selected_content)
        layout.addWidget(self.open_content_button)

        # Set stylesheets for the application
        self.setStyleSheet("""
            QMainWindow {
                background-color: #333333;
            }
            QLabel {
                color: white;
            }
            QPushButton {
                background-color: #4f4f4f;
                color: white;
                font-weight: bold;
            }
            QLineEdit, QComboBox {
                background-color: #FFFFFF;
                color: #000000;
            }
            QListWidget {
                background-color: #FFFFFF;
                color: #000000;
            }
        """)

    def register_user(self):
        username = self.username_input.text()
        role = self.role_input.currentText()
        if not username or not role:
            QMessageBox.warning(self, "Input Error", "Please enter both username and role.")
            return
        user = self.user_factory.create_user(username, role)
        self.data_store.add_user(user)
        self.chat_mediator.register(user)
        QMessageBox.information(self, "Success", f"User {username} registered as {role}")

    def login(self):
        username = self.username_input.text()
        users = self.data_store.get_users()
        for user_data in users:
            if user_data['username'] == username:
                user = self.user_factory.create_user(username, user_data['role'])
                self.session.login(user)
                QMessageBox.information(self, "Success", f"{username} logged in as {user_data['role']}")
                return
        QMessageBox.warning(self, "Login Failed", "User not found")

    def create_course(self):
        if self.session.get_logged_in_user() and self.session.get_logged_in_user().role == 'Teacher':
            course_name = self.course_name_input.text()
            if not course_name:
                QMessageBox.warning(self, "Input Error", "Please enter a course name.")
                return
            course = Course(course_name)
            self.data_store.add_course(course)
            QMessageBox.information(self, "Success", f"Course {course_name} created")
        else:
            QMessageBox.warning(self, "Permission Denied", "Only teachers can create courses")

    def enroll_in_course(self):
        if self.session.get_logged_in_user() and self.session.get_logged_in_user().role == 'Student':
            course_name = self.course_name_input.text()
            courses = self.data_store.get_courses()
            for course_data in courses:
                if course_data['name'] == course_name:
                    course = Course(course_name)
                    enroll_command = EnrollCourseCommand(course, self.session.get_logged_in_user())
                    enroll_command.execute()
                    QMessageBox.information(self, "Success", f"Enrolled in course {course_name}")
                    return
            QMessageBox.warning(self, "Course Not Found", "Course not found")
        else:
            QMessageBox.warning(self, "Permission Denied", "Only students can enroll in courses")

    def add_content_to_course(self):
        if self.session.get_logged_in_user() and self.session.get_logged_in_user().role == 'Teacher':
            course_name = self.course_name_input.text()
            uploaded_file_path = upload_file()
            if uploaded_file_path:
                content_type = os.path.splitext(uploaded_file_path)[1].lower()
                content = {"type": content_type, "path": uploaded_file_path}
                courses = self.data_store.get_courses()
                for course_data in courses:
                    if course_data['name'] == course_name:
                        course = Course(course_name)
                        course.add_content(content)
                        self.data_store.add_course(course)
                        QMessageBox.information(self, "Success", f"Content added to course {course_name}")
                        return
                QMessageBox.warning(self, "Course Not Found", "Course not found")
        else:
            QMessageBox.warning(self, "Permission Denied", "Only teachers can add content to courses")

    def send_message(self):
        sender = self.session.get_logged_in_user()
        if sender:
            receiver_username = self.receiver_input.text()
            message = self.message_input.text()
            if not receiver_username or not message:
                QMessageBox.warning(self, "Input Error", "Please enter both receiver and message.")
                return
            self.chat_mediator.send_message(sender.username, receiver_username, message)
            self.data_store.add_message(sender.username, receiver_username, message)
            QMessageBox.information(self, "Success", "Message sent")
        else:
            QMessageBox.warning(self, "Login Required", "You need to log in to send messages")

    def read_messages(self):
        user = self.session.get_logged_in_user()
        if user:
            messages = self.data_store.get_messages_for_user(user.username)
            self.msg_list.clear()
            if messages:
                for msg in messages:
                    self.msg_list.addItem(f"{msg['timestamp']} - {msg['sender']} to {msg['receiver']}: {msg['message']}")
            else:
                QMessageBox.information(self, "No Messages", "No messages found")
        else:
            QMessageBox.warning(self, "Login Required", "You need to log in to read messages")

    def view_content(self):
        if self.session.get_logged_in_user():
            course_name = self.course_name_input.text()
            if not course_name:
                QMessageBox.warning(self, "Input Error", "Please enter a course name.")
                return
            courses = self.data_store.get_courses()
            self.content_list.clear()
            for course_data in courses:
                if course_data['name'] == course_name:
                    for content in course_data['content']:
                        self.content_list.addItem(content["path"])
                    return
            QMessageBox.warning(self, "Course Not Found", "Course not found")
        else:
            QMessageBox.warning(self, "Login Required", "You need to log in to view content")

    def open_selected_content(self):
        selected_items = self.content_list.selectedItems()
        if selected_items:
            file_path = selected_items[0].text()
            open_file(file_path)
        else:
            QMessageBox.warning(self, "Selection Required", "Please select a content to view")

    def logout(self):
        self.session.logout()
        QMessageBox.information(self, "Success", "Logged out successfully")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = LMSApp()
    window.show()

    # Replace sys.exit(app.exec_()) with a safer alternative for Jupyter
    app.exec_()


ABC to Keerat: Test message
